# Compile data

The purpose of this notebook is to:

1. Combine Tully Runners speed ratings `xlsx` files
2. Fix small Tully Runner's data inaccuracies
3. Convert time to seconds and clean invalid times
4. Merge location data to speed ratings
5. Merge weather data to speed ratings
6. Merge distance data to speed ratings

## Import necessary libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

import requests

## Combine Tully Runners speed ratings `xlsx` files

In [3]:
df = pd.DataFrame()

for year in range(2014, 2020):
    print(year, end=", ")
    
    girls_df = pd.read_excel(f'../data/tully_runners/XCgirls{year}.xlsx')[:-5]
    girls_df['Gender'] = 'Female'
    boys_df = pd.read_excel(f'../data/tully_runners/XCboys{year}.xlsx')[:-5]
    boys_df['Gender'] = 'Male'
    
    df = pd.concat([df, girls_df, boys_df], axis=0)

print(f'Finished! {df.shape}')

2014, 2015, 2016, 2017, 2018, 2019, Finished! (387279, 12)


## Fix small Tully Runner's data inaccuracies

In [4]:
print(f'Old: {df.shape}', end=", ")

df.replace(' ', np.nan, inplace=True)
df["Class"].replace('c', 'C', inplace=True)
df["Section"].replace('08=', '08-', inplace=True)
df["Section"].replace('50-', '05-', inplace=True)

non_NA_cols = ['Speed Rating', 'SR', 'Time', 'Grade', 'Place']

df = df.dropna(subset=non_NA_cols)
df = df[df[non_NA_cols].astype(str).apply(lambda x: x.str.strip().str.len() > 0).all(axis=1)]
    
print(f'New: {df.shape}, Complete!')

Old: (387279, 12), New: (386345, 12), Complete!


## Convert time to seconds and clean invalid times

In [5]:
def time_to_seconds(t):
    if hasattr(t, 'minute') and hasattr(t, 'second'):
        if t.hour and t.hour > 1:
            return t.hour * 60 + t.minute
        return t.hour * 3600 + t.minute * 60 + t.second

    if isinstance(t, str):
        try:
            parts = t.split(':')
            if len(parts) != 2:
                raise ValueError("Expected format 'HH:MM;SS'")
            
            minutes = int(parts[0])
            seconds = int(parts[1].split(';')[0])
            
            return minutes * 60 + seconds

        except (IndexError, ValueError) as e:
            print(f"Error parsing time string: {t}. Reason: {e}")
            return None
            
    print(f"Unrecognized time format: {t}")
    return None

if 'Time' in df.columns:
    time_col_position = df.columns.get_loc('Time')
    df.insert(time_col_position + 1, 'Time (sec)', df['Time'].apply(time_to_seconds))
    print("Inserted Time (sec) column!", end=" --- ")
    
    df = df[df['Time (sec)'] >= 500].copy()
    print(f"Cleaned invalid times!")
    df.drop('Time', axis=1, inplace=True)

Inserted Time (sec) column! --- Cleaned invalid times!


## Merge location data to speed ratings

In [6]:
df_common_sections = df.groupby('Race')['Section'].apply(lambda x: x.mode().iloc[0]).reset_index()
df_section_locations = pd.read_csv("../data/location.csv")
df_race_section_location = df_common_sections.merge(df_section_locations[['Section', 'Latitude', 'Longitude']], on='Section', how='left')
df_race_section_location = df_race_section_location.rename(columns={'Section': 'Race Section'})
df = df.merge(df_race_section_location[['Race', 'Race Section', 'Latitude', 'Longitude']], on='Race', how='left')
print('Location!', list(df.columns))

Location! ['Name', 'Grade', 'Section', 'Class', 'School', 'Race', 'Date', 'Place', 'Time (sec)', 'Speed Rating', 'SR', 'Gender', 'Race Section', 'Latitude', 'Longitude']


## Merge weather data to speed ratings

In [7]:
weather_df = pd.read_csv("../data/weather.csv")
drop_weather_cols = ['Precipitation Coverage', 'Snow', 'Snow Depth']
if 'Unnamed: 0' in weather_df.columns: drop_weather_cols.append('Unnamed: 0')
if 'WeatherData' in weather_df.columns: drop_weather_cols.append('WeatherData')
weather_df.drop(drop_weather_cols, axis=1, inplace=True) # all columns above are unwanted/all null

weather_df['Date'] = weather_df['Date'].astype(str).str.strip()
df['Date'] = df['Date'].astype(str).str.strip()

df = pd.merge(df, weather_df, on=['Date', 'Latitude', 'Longitude'], how='inner')
print('Weather!', list(df.columns))

Weather! ['Name', 'Grade', 'Section', 'Class', 'School', 'Race', 'Date', 'Place', 'Time (sec)', 'Speed Rating', 'SR', 'Gender', 'Race Section', 'Latitude', 'Longitude', 'Temperature', 'Cloud Coverage', 'Wind Speed', 'Precipitation', 'Dew Point', 'Humidity', 'Wind Chill', 'Wind Gust', 'Heat Index', 'Visibility']


## Merge distance data to speed ratings

In [8]:
xls = pd.ExcelFile('../data/distance.xlsx')
sheet_names = xls.sheet_names

distance_data = {}
for sheet in sheet_names:
    distance_data[sheet] = pd.read_excel(xls, sheet_name=sheet)

In [9]:
df['Year'] = pd.to_datetime(df['Date']).dt.year
df = df[(df['Year'] >= 2014) & (df['Year'] <= 2019)]

Sequence match MileSplit race names to Tully Runner's race names. Naming syntax is different per race.

In [10]:
from difflib import SequenceMatcher

def remove_keywords(text, keywords=[]):
    for keyword in keywords:
        text = text.replace(keyword, "")
    return text.strip()

def remove_ending_if_present(s, endings):
    for ending in endings:
        if s.endswith(ending):
            return s[:-len(ending)]
    return s

def find_closest_match(race_name, year):
    kw = ["-V", "-JV", "-Chm", "Class A", "Class B", "Class C", "Class D", "Class", "Championship", "Champ", "Invitational", "Invite", "Invit", "2014", "2015", "2016", "2017", "2018", "2019"]
    race_name = remove_keywords(race_name, kw)
    races = df[df['Year'] == year]['Race']   

    kw2 = ["-Class A", "-Class B", "-Class C", "-Class D", "-VB", "-Lg", "-Md", "-Sm", "-A", "-B", "-C", "-D", "-V", "-JV", "-Chm", " A", " B", " C", " D", " V", " JV", " Chm", " Class A", " Class B", " Class C", " Class D", " Lg", " Md", " Sm"]
    
    max_similarity = 0
    matched_race = None

    for race in races:
        similarity = SequenceMatcher(None, race_name, remove_keywords(race, kw)).ratio()
        if similarity > max_similarity:
            max_similarity = similarity
            matched_race = remove_ending_if_present(race, kw2)
    return matched_race

for year in distance_data:
    print(year, end=", ")
    for race in distance_data[year]["Race"]:
        match = find_closest_match(race, 2014)
        mask = distance_data[year]["Race"] == race
        distance_data[year].loc[mask, "DB-Race"] = match

2014, 2015, 2016, 2017, 2018, 2019, 

Fix sequence matching mistakes

In [11]:
distance_data["2019"][distance_data["2019"] == "NYSPHSAA Championships 2019"]["DB-Race"] = "NY States"
distance_data["2014"][distance_data["2014"] == "Section 7 Championships 2014"]["DB-Race"] = "Section 7-Class"
distance_data["2014"].loc[distance_data["2014"]["Race"] == "Chittenango Modified Invitational 2014", "Year"] = 3.1
distance_data["2015"].loc[distance_data["2015"]["Race"] == "Chittenango Modified Invitational 2015", "Year"] = 3.1

In [12]:
for year in distance_data:
    print(year, end=", ")
    for distance, race in zip(distance_data[year]["Year"], distance_data[year]["DB-Race"]):
        df.loc[(df["Race"].str.startswith(race)) & (df["Year"] == int(year)), "Distance (mi)"] = distance

2014, 2015, 2016, 2017, 2018, 2019, 

In [13]:
df.loc[pd.isna(df["Distance (mi)"]), "Distance (mi)"] = 3.106
df = df[df["Distance (mi)"] != "?"]

In [14]:
mis_labeled = df[(df['Time (sec)'] < 990) & (df['Time (sec)'] > 0) & (df['Speed Rating'] < 130) & (df['Speed Rating'] > 95) & (df['Distance (mi)'] > 2.5)]
df.loc[df["Race"].isin(mis_labeled["Race"].unique()), "Distance (mi)"] = 2.5

df.columns

Index(['Name', 'Grade', 'Section', 'Class', 'School', 'Race', 'Date', 'Place',
       'Time (sec)', 'Speed Rating', 'SR', 'Gender', 'Race Section',
       'Latitude', 'Longitude', 'Temperature', 'Cloud Coverage', 'Wind Speed',
       'Precipitation', 'Dew Point', 'Humidity', 'Wind Chill', 'Wind Gust',
       'Heat Index', 'Visibility', 'Year', 'Distance (mi)'],
      dtype='object')

In [15]:
df.to_csv("../data/compiled.csv", index=False)
print("Saved to compiled.csv!")

Saved to compiled.csv!
